In [96]:
import os
import requests
from datetime import datetime, timedelta
import pandas as pd
import json



root_url = "https://app.ticketmaster.com/discovery/v2/"

In [97]:
def formato_fecha(fecha_str):
        """
        Convierte 'YYYY-MM-DD' a 'YYYY-MM-DDTHH:MM:SSZ'
        """
        return f'{fecha_str}T00:00:00Z'

In [98]:
def extraccion_actual(ciudad, fecha_inicio, API_KEY, fecha_fin, pais='US'):

    url = f'{root_url}events.json'

    params = {
        'apikey' : API_KEY,
        'city' : ciudad,
        'countryCode' : pais,
        'startDateTime': fecha_inicio,
        'endDateTime': fecha_fin,
        'size': 200
    }

    response = requests.get(url, params=params)

    assert response.status_code == 200, "Error en la extracción"
    data = response.json()
    if '_embedded' not in data or 'events' not in data['_embedded']:
      print("No se encontraron eventos")
      return pd.DataFrame()
    
    eventos = data['_embedded']['events']
    
    df = pd.DataFrame(eventos)
    
    return df

In [99]:
API_KEY = os.getenv('TICKETMASTER_API_KEY')
assert API_KEY is not None, "Falta la variable de entorno TICKETMASTER_API_KEY"
fecha_actual = formato_fecha(datetime.now().strftime('%Y-%m-%d'))
manana_obj = datetime.now() + timedelta(days=1)
fecha_fin = formato_fecha(manana_obj.strftime('%Y-%m-%d'))
df = extraccion_actual('New York', fecha_actual, API_KEY, fecha_fin)
df.loc[0]

name                               Harry Potter and the Cursed Child
type                                                           event
id                                                     Z7r9jZ1A7bbfP
test                                                           False
url                 https://www.ticketmaster.com/event/Z7r9jZ1A7bbfP
locale                                                         en-us
images             [{'ratio': '16_9', 'url': 'https://s1.ticketm....
sales              {'public': {'startDateTime': '1900-01-01T18:00...
dates              {'start': {'localDate': '2026-02-16', 'localTi...
classifications    [{'primary': True, 'segment': {'id': 'KZFzniwn...
outlets            [{'url': 'http://www.lyricbroadway.com', 'type...
seatmap            {'staticUrl': 'https://content.resale.ticketma...
ticketing                 {'allInclusivePricing': {'enabled': True}}
nameOrigin                                                    custom
_links             {'self': {'href

In [100]:
df.name.unique()

array(['Harry Potter and the Cursed Child', '11 to Midnight',
       'The Banksy Museum New York!', 'Banksy Museum - Flexiticket',
       'Stranger Things: The First Shadow',
       'The Lion King (New York, NY)', 'Six (New York, NY)',
       'Wicked (NY)', 'Heathers: The Musical',
       'The Gazillion Bubble Show', 'The Play That Goes Wrong', 'Chicago',
       'The Great Gatsby', 'Chicago - The Musical',
       'Gazillion Bubble Show', 'Six: The Musical',
       'Julie Benko & Euphonic Gumbo at Birdland',
       'Anschauung Vol 1 (Sense-Perception) ',
       'Erik Vincent Huey feat: Eric Ambel',
       'Vince Giordano and the Nighthawks', "Jamie Allan's Amaze",
       'Romy and Michele The Musical', 'Jamie Allan',
       'Julie Benko & Euphonic Gumbo Mardi Gras Spectacular',
       'Month Offline Exhibition '], dtype=object)

In [101]:
df.loc[0, "sales"]

{'public': {'startDateTime': '1900-01-01T18:00:00Z',
  'startTBD': False,
  'startTBA': False,
  'endDateTime': '2026-02-17T00:00:00Z'}}

In [102]:
df = df[["name",  "dates", "_embedded"]]
df.loc[15, "dates"]

{'access': {'startDateTime': '2025-08-06T17:24:34Z',
  'startApproximate': False,
  'endDateTime': '2026-02-17T00:00:00Z',
  'endApproximate': False},
 'start': {'localDate': '2026-02-16',
  'localTime': '17:00:00',
  'dateTime': '2026-02-16T22:00:00Z',
  'dateTBD': False,
  'dateTBA': False,
  'timeTBA': False,
  'noSpecificTime': False},
 'end': {'localTime': '17:00:00',
  'dateTime': '2026-02-17T00:00:00Z',
  'approximate': False,
  'noSpecificTime': False},
 'timezone': 'America/New_York',
 'status': {'code': 'onsale'},
 'spanMultipleDays': False}

In [103]:
# 1. Hora de inicio: usamos get() porque si falta dateTime, x['start'] daría error
df['hora_inicio'] = df['dates'].apply(lambda x: x.get('start', {}).get('dateTime'))

# 2. Hora final: corregimos el acceso interno
df['hora_final'] = df['dates'].apply(lambda x: x.get('end', {}).get('dateTime') if 'end' in x else None)

# 3. Dirección, CP y Parking: aquí el riesgo es que 'venues' sea una lista vacía
def extraer_venue_info(embedded):
    # Obtenemos la lista de venues, si no hay, devolvemos una lista con un diccionario vacío [{}]
    venues = embedded.get('venues', [{}])
    v = venues[0] # Cogemos el primer recinto
    
    return pd.Series({
        'direccion': v.get('address', {}).get('line1'),
        'CP': v.get('postalCode'),
        'parking': "Yes" if "parkingDetail" in v else "No"
    })

# Aplicamos la función a la columna _embedded y unimos los resultados
df[['direccion', 'CP', 'parking']] = df['_embedded'].apply(extraer_venue_info)

# 4. Limpieza final de columnas originales
df = df.drop(["dates", "_embedded"], axis=1)

In [104]:
df['hora_inicio'] = pd.to_datetime((df['hora_inicio']))
df['hora_final'] = pd.to_datetime((df['hora_final']))

In [105]:
df['hora_inicio'] = df['hora_inicio'].dt.tz_convert('America/New_York')
df['hora_final'] = df['hora_final'].dt.tz_convert('America/New_York')

In [106]:
df['hora_inicio'] = df['hora_inicio'].dt.strftime('%H:%M')
df['hora_final'] = df['hora_final'].dt.strftime('%H:%M')

In [107]:
df

,name,hora_inicio,hora_final,direccion,CP,parking
0,Harry Potter and the Cursed Child,19:00,NaN,213 W 42nd St,10036,No
1,11 to Midnight,14:00,NaN,126 Second Ave.,10003,Yes
2,The Banksy Museum New York!,10:00,12:00,277 Canal St,10013,No
3,The Banksy Museum New York!,10:30,12:30,277 Canal St,10013,No
4,The Banksy Museum New York!,12:00,14:00,277 Canal St,10013,No
5,The Banksy Museum New York!,11:00,13:00,277 Canal St,10013,No
6,The Banksy Museum New York!,11:30,13:30,277 Canal St,10013,No
7,The Banksy Museum New York!,17:30,19:30,277 Canal St,10013,No
8,The Banksy Museum New York!,18:00,20:00,277 Canal St,10013,No
9,The Banksy Museum New York!,16:00,18:00,277 Canal St,10013,No
